# Use h5py to build sample 'target' files

In [1]:
import h5py
import pystare as ps
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.tri as tri

import cartopy
import cartopy.crs as ccrs

In [2]:
f = h5py.File('MYD09.A2019003_hacked_stare.h5', 'r')

In [3]:
f.keys()

<KeysViewHDF5 ['Latitude', 'Longitude', 'Stare_Index', 'X', 'Y']>

### Determine the lat/lon 'box' for the base data

In [4]:
lat = f['Latitude'][:]
print(lat.max(), lat.min())


32.296478271484375 32.27122116088867


In [5]:
lon = f['Longitude'][:]
print(lon.max(), lon.min())


-98.83241271972656 -98.89727783203125


### Build a target file/dataset the overlaps to the north and west

In [6]:
# Open/create the file

f = h5py.File('Target_overlap_nw.h5','w')

# numpy.arange([start, ]stop, [step, ], dtype=None) -> numpy.ndarray
# numpy.reshape(a, newshape, order='C')

v = np.arange(100, dtype=np.int16) # vector of 0..99
v = np.reshape(v, (10,10)) # 2D 10x10

dset = f.create_dataset("values", data=v)



In [7]:
lat_v = np.arange(32.315, 32.342, 0.003) # 10 values

# Use column_stack to build a matrix where all the X-axis values are
# the same and the Y-axis values vary 

lat_m = np.column_stack((lat_v, lat_v, lat_v, lat_v, lat_v, lat_v, lat_v, lat_v, lat_v, lat_v))

f.create_dataset("Latitude", data=lat_m)

<HDF5 dataset "Latitude": shape (10, 10), type "<f8">

In [8]:
lon_v = np.arange(-98.93, -98.867, 0.007)

# row_stack makes the values vary along the x-axis

lon_m = np.row_stack((lon_v, lon_v, lon_v, lon_v, lon_v, lon_v, lon_v, lon_v, lon_v, lon_v))

f.create_dataset("Longitude", data=lon_m)

<HDF5 dataset "Longitude": shape (10, 10), type "<f8">

In [9]:
# Close the new file, write data
f.close()

### Build the STARE index sidecar files

To do this, run `./build_sidecar -v -r -e "_stare_res.h5" Target_overlap_nw.h5 Latitude Longitude` and 
`./build_sidecar -v -e "_stare.h5" Target_overlap_nw.h5 Latitude Longitude` so there are both the level 27
and level 27 with resolution bits set.

### Plot the Base and Target dataset's STARE indices